In [1]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.feature_selection import SelectFromModel

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt

In [2]:
#Import soccer data(database.sqlite) using pandas.
import pandas as pd
import sqlite3
db = 'database.sqlite'
conn = sqlite3.connect(db)
cur = conn.cursor()

In [3]:
#Load the values from the attributes X= ['strength', 'stamina'] and Y= ['jumping'] from table Player_Attributes.

df = pd.read_sql("SELECT strength, stamina, jumping FROM Player_Attributes", conn)
df.fillna(11, inplace=True)

df.head()

,strength,stamina,jumping
0,76.0,54.0,58.0
1,76.0,54.0,58.0
2,76.0,54.0,58.0
3,76.0,54.0,58.0
4,76.0,54.0,58.0


In [4]:
x = df[['strength', 'stamina']].values
y = df[['jumping']].values
X_train, X_test, y_train, y_test = train_test_split(x, y)


In [5]:
#Using the above X and Y, apply GridSearch on DecisionTreeClassifier and fetch the best params. 
from sklearn import metrics

model = DecisionTreeClassifier()
dt_mod = model.fit(X_train, y_train)
dt_preds = dt_mod.predict(X_test)
accuracy = metrics.accuracy_score(y_test, dt_preds)
accuracy

0.16460484835308187

In [12]:
#apply gridsearch

model.get_params()


{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [30]:
#params_grid = {'criterion':['gini','entropy'],'splitter':['best','random'],'max_depth':[1,5,10,20,30,40,50],'random_state':[0,5,10,20,30,40,50]}

params_grid = {'criterion':['gini','entropy'], 'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50], 'splitter':["best", "random"], 'random_state':[0,1,2,4,6,8,10,12,14,16,20,40,42]}


In [31]:
grid_search = GridSearchCV(DecisionTreeClassifier(), params_grid)

In [32]:
grid_search.fit(X_train, y_train)

C:\Users\benso\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 20, 30,
                                       40, 50],
                         'random_state': [0, 1, 2, 4, 6, 8, 10, 12, 14, 16, 20,
                                          40, 42],
                         'splitter': ['best', 'random']})

In [34]:
#Using this best params (from 3.1) and the selected features (from 3.3), rebuild the DecisionTreeClassifier model and generate the classification_report.

print("best score: " + str(grid_search.best_score_))
print("test score: " + str(grid_search.score(X_test, y_test)))
decision_tree_best_params = grid_search.best_params_
print("best parameters: " + str(decision_tree_best_params))



best score: 0.1596428582289639
test score: 0.16462658984672246
best parameters: {'criterion': 'entropy', 'max_depth': 20, 'random_state': 4, 'splitter': 'best'}


In [35]:
#re-do with best parameters

model = DecisionTreeClassifier(criterion='gini',max_depth=20,random_state=4,splitter='best')
dt_mod = model.fit(X_train, y_train)
dt_preds = dt_mod.predict(X_test)
accuracy = metrics.accuracy_score(y_test, dt_preds)
accuracy

#same as before....

0.161169692357865

In [6]:
#repeat for SVM
from sklearn import svm
from sklearn.svm import LinearSVC

#from sklearn.preprocessing import MinMaxScaler
#scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
#X_train = scaling.transform(X_train)
#X_test = scaling.transform(X_test)

from sklearn import preprocessing
X_train = preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)

model = LinearSVC(random_state=0, tol=1e-5)
svm_mod = model.fit(X_train, y_train.ravel()) 

#model = svm.SVC()
#dt_mod = model.fit(X_train, y_train.ravel())
svm_preds = svm_mod.predict(X_test)
accuracy = metrics.accuracy_score(y_test, svm_preds)
accuracy


In [8]:
#repeat for LogisticRegression

from sklearn import metrics
from sklearn import preprocessing
X_train = preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)

model = LogisticRegression()
dt_mod = model.fit(X_train, y_train.ravel())
dt_preds = dt_mod.predict(X_test)
accuracy = metrics.accuracy_score(y_test, dt_preds)
accuracy

C:\Users\benso\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.053223176432220895

In [9]:
model.get_params()


{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [17]:
params_grid = {'C':[1.0,2.0,3.0], 'max_iter':[100,200,300]}

In [18]:
grid_search = GridSearchCV(LogisticRegression(), params_grid)

In [19]:
grid_search.fit(X_train, y_train.ravel())

C:\Users\benso\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
C:\Users\benso\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\benso\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CON

KeyboardInterrupt: 

In [7]:
#repeat for KNeighborsClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics


model = KNeighborsClassifier()

k_neigh = model.fit(X_train, y_train)

k_preds = k_neigh.predict(X_test)
accuracy = metrics.accuracy_score(y_test, k_preds)
accuracy

C:\Users\benso\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.11312099141211002

In [8]:
model.get_params()


{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [9]:
params_grid = {'algorithm':['auto','ball_tree','kd_tree'], 'n_neighbors':[2,5,10,20]}

In [10]:
grid_search = GridSearchCV(KNeighborsClassifier(), params_grid)

In [11]:
grid_search.fit(X_train, y_train.ravel())

C:\Users\benso\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree'],
                         'n_neighbors': [2, 5, 10, 20]})

In [12]:
print("best score: " + str(grid_search.best_score_))
print("test score: " + str(grid_search.score(X_test, y_test)))
K_neighbors_best_params = grid_search.best_params_
print("best parameters: " + str(K_neighbors_best_params))

best score: 0.12534873906768934
test score: 0.12947059462985108
best parameters: {'algorithm': 'auto', 'n_neighbors': 20}


In [ ]:
#two of the classifiers took more than 3 hours to run, so the data is inconclusive. Based on all of this, I would say that the KNeighborsClassifier was the best classifier, where the best parameters are the following:
#'auto' algorithm, with 20 neighbors.

